<h1 style="text-align: center;">ETL</h1>

## Soils

In [2]:
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# --- Curated List of Willamette Valley Soil Series ---
willamette_valley_soils = [
    {
        "name": "Jory",
        "description": "Deep, well-drained red clay soil formed from ancient volcanic basalt. Famous in the Dundee Hills, known for producing elegant and complex Pinot Noir.",
        "parent_material": "Volcanic (Basalt)"
    },
    {
        "name": "Willakenzie",
        "description": "Moderately deep, well-drained silty clay loam formed from marine sedimentary rock. Tends to hold more water than volcanic soils, often resulting in darker fruit profiles.",
        "parent_material": "Marine Sediment"
    },
    {
        "name": "Nekia",
        "description": "Similar to Jory but shallower, well-drained clay loam over basalt bedrock. Often found at higher elevations.",
        "parent_material": "Volcanic (Basalt)"
    },
    {
        "name": "Laurelwood",
        "description": "Deep, well-drained silt loam formed from wind-deposited loess over basalt. Predominant in the Chehalem Mountains AVA.",
        "parent_material": "Loess (Windblown Silt)"
    },
    {
        "name": "Bellpine",
        "description": "A silty clay loam soil derived from marine sedimentary rock, similar to Willakenzie but often shallower. Common in the Eola-Amity Hills.",
        "parent_material": "Marine Sediment"
    },
    {
        "name": "Chehulpum",
        "description": "A silty clay loam found on terraces, formed from a mix of sedimentary and volcanic alluvium.",
        "parent_material": "Alluvium"
    },
    {
        "name": "Woodburn",
        "description": "A deep, moderately well-drained silt loam found on valley floors, formed from alluvial fan deposits. Less common for high-quality vineyards but part of the valley's soil-scape.",
        "parent_material": "Alluvium"
    },
    {
        "name": "Hazelair",
        "description": "A clayey soil derived from weathered tuff and breccia, found in the Eola-Amity Hills.",
        "parent_material": "Volcanic (Tuff)"
    },
    {
        "name": "Melbourne",
        "description": "A moderately deep, well-drained silty clay loam formed in material weathered from sandstone and siltstone.",
        "parent_material": "Marine Sediment"
    },
    {
        "name": "Goodman",
        "description": "Well-drained loam derived from igneous rock, often found on foothills and mountains.",
        "parent_material": "Volcanic (Igneous)"
    },
    {
        "name": "Steiwer",
        "description": "Shallow, well-drained loam overlying basalt bedrock, often found on steep slopes.",
        "parent_material": "Volcanic (Basalt)"
    },
    {
        "name": "Saum",
        "description": "A silty clay loam that is somewhat poorly drained, typically found in lower-lying areas and depressions.",
        "parent_material": "Alluvium"
    },
    {
        "name": "Willamette",
        "description": "Very deep, well‑drained silty loam formed in glaciolacustrine (lake) deposits—extensive on valley terraces and floors.",
        "parent_material": "Glaciolacustrine Sediment"
    },
    {
        "name": "Dupee",
        "description": "Sibling series to Willakenzie, often occurring together—well‑drained silty clay loam over marine sedimentary bedrock.",
        "parent_material": "Marine Sediment"
    },
    {
        "name": "Carlton",
        "description": "A marine‑derived silty clay loam similar to Willakenzie and Dupee; often mapped in vineyards on marine sedimentary hills.",
        "parent_material": "Marine Sediment"
    },
    {
        "name": "Panther",
        "description": "Another marine‑sedimentary sibling to Willakenzie—well‑drained silty clay loam common in the foothills.",
        "parent_material": "Marine Sediment"
    },
    {
        "name": "Wellsdale",
        "description": "Fine marine sedimentary loam (sandstone with trace oceanic salts), found in Chehalem Mountains, imparting saline minerality.",
        "parent_material": "Marine Sediment"
    },
    {
        "name": "Kinton",
        "description": "Fine loess silt loam over basalt, found primarily in the Tualatin Hills region.",
        "parent_material": "Loess (Windblown Silt)"
    },
    {
        "name": "Cornelius",
        "description": "Fine loess silt loam over basalt, common in northern Willamette Valley, contributing to deep, fertile profiles.",
        "parent_material": "Loess (Windblown Silt)"
    }
]

# --- Database Connection ---
dotenv_path = '../.env'
load_dotenv(dotenv_path=dotenv_path)

DATABASE_URL = os.getenv('DATABASE_URL')

if not DATABASE_URL:
    raise ValueError("DATABASE_URL not found in .env file. Make sure .env file is in the 'backend' directory.")

engine = create_engine(DATABASE_URL)

# --- Ingest into Database ---
print("Starting ingestion of curated soil list...")
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            total_ingested = 0
            for soil in willamette_valley_soils:
                stmt = text("""
                    INSERT INTO soils (name, description, parent_material)
                    VALUES (:name, :description, :parent_material)
                    ON CONFLICT (name) DO UPDATE SET
                        description = EXCLUDED.description,
                        parent_material = EXCLUDED.parent_material;
                """)
                
                result = connection.execute(stmt, {
                    "name": soil['name'],
                    "description": soil['description'],
                    "parent_material": soil['parent_material']
                })

                if result.rowcount > 0:
                    total_ingested += 1

            print(f"Ingestion complete. Processed {len(willamette_valley_soils)} soils. Added or updated {total_ingested} records.")
            # The transaction is automatically committed here
        except Exception as e:
            print(f"An error occurred: {e}")
            # The transaction is automatically rolled back
            transaction.rollback()
            raise

print("\n Finished.")

Starting ingestion of curated soil list...
Ingestion complete. Processed 19 soils. Added or updated 19 records.

 Finished.


## Varietals

In [ ]:
import pandas as pd
import sqlalchemy
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")
engine = sqlalchemy.create_engine(DATABASE_URL)

# Varietals table, to add more data, first delete current data to prevent duplicates and then re-run this cell
varietals_data = {
    "Pinot Noir": "The quintessential Willamette Valley grape. It presents a silky, elegant texture balanced by vibrant acidity. Expect a complex aromatic profile of wild strawberry, Rainier cherry, and damp forest floor, with a subtle spice of white pepper. The volcanic Jory soils often impart a distinct stony minerality, while marine sedimentary soils can add a savory, umami-like depth.",
    "Pinot Gris": "A textural and aromatic delight, often crafted in an Alsatian-inspired style. It reveals layers of ripe pear, quince, and beeswax, with a spicy hint of ginger. A brief period of skin contact lends a beautiful copper hue and a pleasant bitter note of orange peel, adding to its complexity.",
    "Chardonnay": "A grape of two distinct personalities in the Valley. The 'Lean' style, often from the Eola-Amity Hills, offers crisp green apple, wet flint, and lemon pith with laser-like acidity. The 'Rich' style, typical of the Dundee Hills, delivers notes of baked apple, brioche, and toasted hazelnut with a rounder, creamier mid-palate.",
    "Riesling": "A noble grape with remarkable aging potential. In its youth, it bursts with notes of peach skin and a hint of kerosene, which evolves into complex honeycomb and petrol notes over time. Dry expressions possess a searing acidity, while late-harvest styles are luscious with apricot nectar.",
    "Gamay": "Vibrant and juicy, this grape is all about immediate pleasure. It explodes with flavors of cranberry, pomegranate, and rose hip, grounded by a subtle black tea leaf earthiness. Carbonic maceration often lends a characteristic bubblegum lift and softens the tannins, making it irresistibly quaffable.",
    "Syrah": "A testament to cool-climate intensity. This grape offers a brooding bouquet of blackberry compote, violets, and savory notes of smoked meat and black olive. The persistent winds of the Van Duzer Corridor impart a signature peppery grip and firm, chewy tannins.",
    "Pinot Blanc": "A varietal of quiet power and finesse. It whispers notes of lemon curd, raw almond, and a distinct saline minerality reminiscent of sea spray. It is often blended with Chardonnay to enhance textural weight and complexity.",
    "Dolcetto": "An Oregonian take on a Piedmont classic. It features a core of bitter cherry, interwoven with notes of tar, licorice, and blueberry skin. Compared to its Italian counterpart, the Willamette Valley version often showcases brighter acidity and a unique iron oxide grit from the local soils.",
    "Tempranillo": "Feral, savory, and complex. This Spanish grape reveals a rustic profile of sour plum, tobacco leaf, and worn leather. It demands sun-drenched, south-facing slopes to achieve full ripeness and express its sun-baked character.",
    "Lagrein": "Inky, brooding, and powerful. This Northern Italian varietal is characterized by intense notes of black raspberry, dark cocoa powder, and a ferrous hint of beef blood. The tannins are famously robust, often described as 'chewing on grape seeds.'",
    "Carmine": "A fascinating hybrid with a bold personality. It combines the structure of Cabernet Sauvignon with the rustic charm of Carignan, resulting in a wine with notes of cranberry sauce, bell pepper, and dark chocolate.",
    "Grüner Veltliner": "Austria's star grape, finding a happy home in Oregon. It is defined by its signature white pepper and radish-like crunch, complemented by flavors of green pear, lentil, and lime zest. The local Jory soils contribute a smoky, flinty nuance.",
    "Müller-Thurgau": "A charming, off-dry white perfect for casual sipping. It delights with aromatic notes of lychee, elderflower, and bruised apple. Its naturally low alcohol content makes it a light and refreshing picnic wine.",
    "Albariño": "A burst of seaside freshness. This Spanish varietal is defined by its saline minerality, evoking sea spray on a coastal breeze. Flavors of white peach and lemongrass are carried on a wave of crisp acidity, making it exceptionally refreshing.",
    "Arneis": "A varietal of bitter elegance from Piedmont. It offers a sophisticated bouquet of fennel pollen, chamomile, and blanched almond. The finish is characteristically bitter, reminiscent of a peach pit, which adds to its allure.",
    "Pinot Meunier": "Primarily a sparkling wine grape, it shines with notes of red apple skin and freshly baked brioche. It contributes to a creamy, persistent mousse, creating a wine of great texture and charm.",
    "Gewürztraminer": "An aromatic explosion in the glass. It offers an unmistakable bouquet of lychee and rosewater, with a rich, slightly oily texture that coats the palate. Willamette Valley versions are often fermented dry, showcasing the grape's full aromatic potential.",
    "Pinot Noir Précoce": "An early-ripening clone, considered an ancestor of Pinot Noir. It delivers a wilder, more rustic expression of its descendant, with intense notes of wild strawberry and blood orange.",
    "Blaufränkisch": "An Austrian red with a savage edge. It presents a deep profile of blueberry liqueur and graphite, supported by fierce, structured tannins that demand attention.",
    "Sauvignon Blanc": "A nod to the restrained style of the Loire Valley. It offers crisp notes of gooseberry and freshly cut grass, with a focus on minerality and elegance rather than overt tropical fruit."
}

df_varietals = pd.DataFrame(list(varietals_data.items()), columns=['name', 'description'])
df_varietals.to_sql('varietals', engine, if_exists='append', index=False)

print("Varietals table populated successfully.")

Varietals table populated successfully.
